In [1]:
# In this script, we will read in the downwelling and upwelling surface radiation
# to produce the albedo. Then, we can multiply changes in albedo by the albedo kernels,
# resulting in the TOA radiative perturbation from changes in albedo.

# By: Ty Janoski
# updated: 06.15.20

In [2]:
# Import statements

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import xesmf as xe
%matplotlib inline
%config InlineBackend.figure_format = "png"

In [4]:
# Load in CAM5 abledo kernels
ds_ker = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/\
CAM5_kernels/alb.kernel.nc')
lats = ds_ker.lat
lons = ds_ker.lon

# create a dataset with lat/lons for regridding
ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )

In [5]:
diff_list = []
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for mod in models:
    print(mod)
    # read in last 50 years of piControl
    # rsus = upward SW at surf, rsds = downward SW at surf
    rsds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsds_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).rsds.isel(time=
                                                                          slice(-600,None))
    if(mod!='CCSM4'):
        rsus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/rsus_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).rsus.isel(time=
                                                                          slice(-600,None))
    else:
        rsus = xr.open_dataset('http://strega.ldeo.columbia.edu:81/CMIP5/.byScenario/'+
                               '.piControl/.atmos/.mon/.rsus/.CCSM4/.r1i1p1/.rsus/dods',
                               decode_times=False)['rsus'].isel(
            T=slice(-600,None)).rename({'T':'time'})
        rsus['time'] = rsds['time']
    # albedo is up/down SW
    albedo = rsus/(rsds.where(rsds>1e-3))
    
    # make monthly climatology
    clim = albedo.groupby(albedo.time.dt.month).mean(dim='time').compute()
    
    # repeat process for abrupt4xCO2
    
    rsus = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsus_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).rsus.isel(time=
                                                                          slice(None,1800))
    rsds = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/rsds_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).rsds.isel(time=
                                                                          slice(None,1800))
    albedo = rsus/(rsds.where(rsds>1e-3))
    
    # take difference between abrupt4xCO2 and monthly climatology
    regridder = xe.Regridder(albedo,ds_out,'bilinear',periodic=True,reuse_weights=True)

    diff = regridder((albedo - np.tile(clim,(150,1,1))).compute())
    diff_list.append(diff)
        
    all_sky = (diff * np.tile(ds_ker.FSNT,(150,1,1)) * 100).fillna(0)
    clear_sky = (diff * np.tile(ds_ker.FSNTC,(150,1,1)) * 100).fillna(0)
    
    # save output
    out = xr.merge([all_sky.rename('albedo_SWAS'),clear_sky.rename('albedo_SWCS')])
    out.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+
                  '_albedo.nc')

ACCESS1-0


/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Reuse existing file: bilinear_145x192_192x288_peri.nc
ACCESS1-3


/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Reuse existing file: bilinear_145x192_192x288_peri.nc
CNRM-CM5


...

/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Reuse existing file: bilinear_90x144_192x288_peri.nc
MIROC-ESM


/home/tylerj/miniconda3/envs/xesmf_env/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Reuse existing file: bilinear_64x128_192x288_peri.nc
FGOALS-g2


KeyboardInterrupt: 